# NeRF

## 0. Hyperparameters

In [12]:
import argparse

args = {
    # For NeRF NNs and Optimizer
    "coarse_net_width": 256,
    "coarse_net_depth": 8,
    "fine_net_width": 256,
    "fine_net_depth": 8,
    "learning_rate": 5e-4,
}
args = argparse.Namespace(**args)

## 1. Model

In [13]:
import torch


class NeRF(torch.nn.Module):
    def __init__(self, net_width: int, net_depth: int, input_channel: int, output_channel: int,
                 input_channel_views: int, use_view: bool, skips: list):
        super(NeRF, self).__init__()
        self.net_width = net_width
        self.net_depth = net_depth
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.input_channel_views = input_channel_views
        self.use_view = use_view
        self.skips = skips

        self.points_layers = torch.nn.ModuleList(
            [torch.nn.Linear(in_features=input_channel, out_features=net_width)] + [
                torch.nn.Linear(in_features=net_width, out_features=net_width) if i not in skips else torch.nn.Linear(
                    in_features=(net_width + input_channel), out_features=net_width) for i in range(net_depth - 1)])
        self.views_layers = torch.nn.ModuleList(
            [torch.nn.Linear(in_features=input_channel_views + net_width, out_features=net_width // 2)])
        if use_view:
            self.feature_layer = torch.nn.Linear(net_width, net_width)
            self.alpha_layer = torch.nn.Linear(net_width, 1)
            self.rgb_layer = torch.nn.Linear(net_width // 2, 3)
        else:
            self.output_linear = torch.nn.Linear(net_width, output_channel)

    def forward(self, x):
        pass


coarse_model = NeRF(args.coarse_net_width, args.coarse_net_depth, 3, 4, 8, True, [4])
grad_vars = list(coarse_model.parameters())
optimizer = torch.optim.Adam(params=grad_vars, lr=args.learning_rate, betas=(0.9, 0.999))

2. 